In [1]:
import os

In [3]:
diags = [x for x in os.listdir("data/train/") if x.startswith("dialogues_pt")]
diags

['dialogues_pt_015.json',
 'dialogues_pt_005.json',
 'dialogues_pt_013.json',
 'dialogues_pt_014.json',
 'dialogues_pt_017.json',
 'dialogues_pt_003.json',
 'dialogues_pt_008.json',
 'dialogues_pt_012.json',
 'dialogues_pt_006.json',
 'dialogues_pt_007.json',
 'dialogues_pt_002.json',
 'dialogues_pt_004.json',
 'dialogues_pt_010.json',
 'dialogues_pt_011.json',
 'dialogues_pt_009.json',
 'dialogues_pt_001.json',
 'dialogues_pt_016.json']